# RAG


# Overview

검색 증강 생성(Retrieval Augmented Generation)은 대규모 언어 모델이 지식을 생성할 때 외부 지식원을 활용하는 기술을 말합니다.

쉽게 설명하자면, 언어 모델이 문장을 생성할 때 관련된 정보를 인터넷이나 데이터베이스에서 찾아 활용하는 것입니다. 이렇게 하면 모델이 가진 지식의 한계를 넘어 더 정확하고 상세한 내용을 생성할 수 있습니다.

예를 들어 "파리의 인구는 얼마인가?"라는 질문에 대해, 모델은 위키피디아 등에서 파리 인구 통계를 찾아 그 정보를 활용하여 대답할 수 있습니다. 단순히 학습된 지식만으로는 정확한 최신 정보를 제공하기 어렵지만, 외부 지식원을 참고하면 더 나은 결과를 낼 수 있습니다.

이 과정에서는 앞서 생성한 `movie_semantic` 인덱스를 Knowledge Base로 Amazon Bedrock의 Claude V3 Sonnet 모델을 사용하여 환각을 제거하고 정확한 답변을 생성해봅니다.


# 사전준비

이번 단계를 진행하기 위해서는 [시맨틱 검색 단계](./02.semantic_search.ipynb)를 필수적으로 완료하셔야 합니다. Amazon OpenSearch Service로의 연결은 [시맨틱 검색 단계](./02.semantic_search.ipynb)와 동일하게 수행합니다.


In [1]:
%store -r model_id
%store -r index_name

패키지를 설치합니다.


## 환각 발생시키기

RAG를 구성하지 않고 바로 Amazon Bedrock Claude Sonnet V3에게 다음과 같은 질문을 던져 봅니다.


In [6]:
import textwrap

In [3]:
query_text = "건축학개론 내용과 평점은?"
print(query_text)

건축학개론 내용과 평점은?


In [2]:
import os
from dotenv import dotenv_values

env_vars = dotenv_values('.env_api')
os.environ["OPENAI_API_KEY"] = env_vars.get("OPENAI_API_KEY")

In [9]:
import sys

sys.stdout

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(
    model_name='gpt-4o-mini-2024-07-18',
    max_tokens=2048,
    temperature=0,
    streaming=True,
)

messages = [HumanMessage(content=query_text)]

print(textwrap.fill(llm.invoke(messages).content, 80))

《건축학개론》은 2012년에 개봉한 한국의 로맨스 드라마 영화로, 감독은 이재규입니다. 이 영화는 건축학과 학생들의 첫사랑과 성장 이야기를
중심으로 전개됩니다. 주인공인 남자와 여자는 대학 시절 서로에게 끌리지만, 여러 가지 이유로 사랑이 이루어지지 못하고 시간이 흐른 후 다시
만나는 이야기를 담고 있습니다.  영화는 감성적인 스토리와 아름다운 영상미로 많은 관객들에게 사랑받았으며, 특히 첫사랑의 아련한 감정을 잘
표현했다는 평가를 받았습니다.   평점은 다양한 플랫폼에서 다르게 나타날 수 있지만, 일반적으로 7점대에서 8점대의 평점을 기록하고 있습니다.
관객과 평론가들 모두에게 긍정적인 반응을 얻은 작품입니다.   더 구체적인 평점이나 리뷰는 영화 관련 사이트에서 확인하실 수 있습니다.


gpt-4o-mini는 한국 영화 건축학 개론에 대한 답을 모르기 때문에 엉뚱한 답변을 하게 됩니다.


## RAG을 통해 환각 없애기


이제 OpenSearch에 저장된 정보를 컨텍스트로 전달하여 환각을 없애보겠습니다. 여기서는 이전 단계에서 사용한 [하이브리드 검색 기법](./03.hybrid_search.ipynb)을 활용해보도록 하겠습니다


## OpenSearch 클라이언트 연결


이전 과정에서 수행했던 것과 동일한 방법으로 OpenSearch 도메인에 연결합니다.


가져온 정보를 바탕으로 OpenSearch 도메인에 연결합니다.


In [10]:
from opensearchpy import OpenSearch, RequestsHttpConnection
import json

# OpenSearch 연결 설정
base_url = "https://localhost:9200/_plugins/_ml"
host = 'localhost'
port = 9200
auth = ('admin', 'TestUser2@')  # 초기 설정한 어드민 비밀번호 사용

aos_client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=False,
    ssl_show_warn=False,
)

인덱스명과 모델 아이디를 설정합니다.


In [ ]:
import requests

# search_model = {"query": {"match": {"name": "OpenSearch-Cohere"}}, "size": 10}

# response = requests.get(
#     "https://" + aos_host + "/_plugins/_ml/models/_search", auth=auth, json=search_model
# )
# model_info = json.loads(response.text)
# model_id = model_info["hits"]["hits"][0]["_id"]

# index_name = "movie_semantic"

## 하이브리드 검색 함수 구현

이전 [하이브리드 검색 단계](https://www.notion.so/Hybrid-Search-9b923c1a7b2e4d1697768c3385ea47d0?pvs=21)에서 사용한 `hybrid_search` 함수를 다음과 같이 변경합니다. 가장 큰 변경 부분은 검색된 결과를 LangChain의 Document 객체로 변환해서 반환한다는 점입니다. 이렇게 생성된 **`Document`** 객체들을 기반으로 LangChain은 다양한 작업을 수행할 수 있습니다.


In [11]:
from langchain.schema import Document
import pandas as pd


def hybrid_search(
    query_text,
    keyword_weight=0.3,
    semantic_weight=0.7,
):
    print(query_text)
    query = {
        "size": 10,
        "_source": {"exclude": ["text", "vector_field"]},
        "query": {
            "hybrid": {
                "queries": [
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["title", "plot", "genre", "main_act", "supp_act"],
                        }
                    },
                    {
                        "neural": {
                            "vector_field": {
                                "query_text": query_text,
                                "model_id": model_id,
                                "k": 30,
                            }
                        }
                    },
                ]
            }
        },
        "search_pipeline": {
            "description": "Post processor for hybrid search",
            "phase_results_processors": [
                {
                    "normalization-processor": {
                        "normalization": {"technique": "min_max"},
                        "combination": {
                            "technique": "arithmetic_mean",
                            "parameters": {"weights": [keyword_weight, semantic_weight]},
                        },
                    }
                }
            ],
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    docs = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["title"],
            hit["_source"]["plot"],
            hit["_source"]["genre"],
            hit["_source"]["rating"],
            hit["_source"]["main_act"],
        ]
        query_result.append(row)

        # LangChain에 Context로 제공하기 위한 Document 객체를 준비합니다.
        metadata = {"score": hit["_score"], "id": hit["_id"]}

        content = {
            "제목": hit["_source"]["title"],
            "장르": hit["_source"]["genre"],
            "평점": hit["_source"]["rating"],
            "줄거리": hit["_source"]["plot"],
            "주연": hit["_source"]["main_act"],
            "조연": hit["_source"]["supp_act"],
        }

        doc = Document(page_content=json.dumps(content, ensure_ascii=False), metadata=metadata)

        docs.append(doc)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "title", "plot", "genre", "rating", "main_act"]
    )
    display(query_result_df)

    return docs

검색 함수가 잘 동작하는지 확인합니다.


In [12]:
docs = hybrid_search(query_text)

print("검색된 문서 총 " + str(len(docs)) + "개")

건축학개론 내용과 평점은?


,_score,title,plot,genre,rating,main_act
0,1.000000,건축학개론,"생기 넘치지만 숫기 없던 스무 살, 건축학과 승민은 '건축학개론' 수업에서 처음 만...",멜로/로맨스,8.67,엄태웅|한가인|이제훈|수지
1,0.220972,투사부일체,"5년 전, 조폭의 신분으로 고등학교에 입학. 학교를 발칵 뒤집어 놓은 계두식이, 이...",코미디|액션,5.59,정준호|김상중|정웅인|정운택
2,0.202281,임금님의 사건수첩,모든 사건은 직접 파헤쳐야 직성이 풀리는 총명한 왕 ‘예종’(이선균). 그를 보좌하...,코미디|모험|액션,7.71,이선균|안재홍
3,0.137774,강철중: 공공의 적 1-1,강동서 강력반 꼴통 형사 강철중(설경구). 5년이 지난 지금도 여전히 사건 현장을 ...,범죄|스릴러|코미디|드라마|액션,8.53,설경구|정재영
4,0.094635,지구를 지켜라!,병구는 외계인으로 인해 지구가 곧 위험에 처할 거라고 믿는다. 이번 개기월식까지 안...,SF|스릴러,8.85,신하균|백윤식
5,0.087059,한반도,남과 북이 통일을 약속하고 그 첫 상징인 경의선 철도 완전 개통식을 추진한다. 그러...,드라마|액션|스릴러,6.36,조재현|차인표|안성기|문성근|강신일
6,0.075363,고사: 피의 중간고사,"수능을 약 2 여일 앞둔 어느 토요일, 전교 1등부터 2 등까지의 모범생들만을 위한...",공포|스릴러,6.82,이범수|윤정희|남규리|김범
7,0.075048,기담,동경 유학 중이던 엘리트 의사 부부 인영(김보경)과 동원(김태우)은 갑작스레 귀국하...,공포,7.48,진구|이동규|김태우|김보경
8,0.055312,울학교 이티,"옹골찬 근육으로 다져진 특 1등급 건강인을 자부하는 천성근, 그는 우리나라 최고의 ...",코미디|드라마,8.99,김수로|이한위|백성현|박보영|이민호|문채원|이찬호|김성령
9,0.000700,작업의 정석,작업계의 대표선수 민준과 지원이 만났다. 선수는 선수를 알아보는 법! 작업계의 고수...,코미디|멜로/로맨스,6.53,손예진|송일국


검색된 문서 총 10개


## RAG을 통한 정확한 답변 받기


### 컨텍스트 기반 질문 응답을 위한 프롬프트 템플릿 생성

LangChain의 **`PromptTemplate`**을 사용하여 컨텍스트 기반 질문 응답을 위한 프롬프트 템플릿을 정의합니다.

1. 컨텍스트와 질문을 입력 변수로 받습니다.
2. 컨텍스트는 **`<context>`** 및 **`</context>`** XML 태그로 묶여 제공됩니다.
3. 답변 시 XML 태그를 포함하지 않도록 지시합니다.
4. 답변 시 가능한 한 많은 내용을 포함하도록 지시합니다.
5. 답변 시 공손하고 예의바른 태도를 유지하도록 지시합니다.
6. 컨텍스트에서 답변을 확실히 찾을 수 없는 경우, "주어진 내용에서 관련 답변을 찾을 수 없습니다."라고 답변하도록 지시합니다.


In [13]:
from langchain import PromptTemplate

prompt_template = """


Human: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the contex as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "주어진 내용에서 관련 답변을 찾을 수 없습니다."


Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

LangChain의 **`load_qa_chain`** 함수를 사용하여 질문 답변 체인을 로드하고, 주어진 문서와 질문에 대한 답변을 생성합니다. 하이브리드 검색으로 찾은 문서 `docs`를 `chain`의 `input_documents`로 제공합니다.


In [14]:
from langchain.chains.question_answering import load_qa_chain
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    model_name='gpt-4o-mini-2024-07-18',
    max_tokens=2048,
    temperature=0,
    streaming=True,
)

chain = load_qa_chain(llm=llm, chain_type="stuff", prompt=PROMPT)

answer = chain.run(input_documents=docs, question=query_text)

print("query: ", query_text)
answer_str = "answer: \n" + answer
print(textwrap.fill(answer_str, 80))

/home/ubuntu/miniconda3/envs/aws_workshop/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  warn_deprecated(
/home/ubuntu/miniconda3/envs/aws_workshop/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_de

query:  건축학개론 내용과 평점은?
answer:  "건축학개론"의 줄거리는 다음과 같습니다: 생기 넘치지만 숫기 없던 스무 살, 건축학과 승민은 '건축학개론' 수업에서 처음 만난
음대생 서연에게 반합니다. 함께 숙제를 하게 되면서 차츰 마음을 열고 친해지지만, 자신의 마음을 표현하는 데 서툰 순진한 승민은 입 밖에 낼 수
없었던 고백을 마음 속에 품은 채 작은 오해로 인해 서연과 멀어지게 됩니다. 15년 만에 그녀를 다시 만난 승민은 서연에게 자신을 위한 집을
설계해달라는 요청을 받게 되고, 함께 집을 완성해 가는 동안 그때의 기억이 되살아나 두 사람 사이에 새로운 감정이 쌓이기 시작합니다.  이
영화의 평점은 8.67입니다.


위와 같이 정확한 답변이 제공되는 것을 확인할 수 있습니다.


In [15]:
%store model_id
%store index_name

Stored 'model_id' (str)
Stored 'index_name' (str)
